In [32]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Flatten
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model

from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Lambda
from keras.layers import Activation
from matplotlib import pyplot
from keras import backend

from keras.layers import *
from keras import *

import numpy as np

https://machinelearningmastery.com/semi-supervised-generative-adversarial-network/

In [90]:
X_train, index = np.load('processed/data/X.npy'), np.load('processed/data/index.npy')
X_train.shape, index.shape

((2718, 100, 25), (2718,))

In [170]:
y_train = []
for i in range(len(index)):
    if index[i] < 1279823200 and (index[i] + 100 * 21600) > 1278168000:
        y_train.append(1)
    else:
        y_train.append(0)
y_train = np.array(y_train)

In [172]:
sum(y_train)

177

# Discriminator

## Separate Discriminator Models With Shared Weights

1-D convolutional layer for both Critics, with the intention of capturing local temporal features that can determine how anomalous a sequence is.


In [53]:
# What is the best way combine the surpvised + unsurpervised ?

def define_discriminator():
    in_shape = (100, 25)
    in_image = Input(shape=in_shape)
    fe = Conv1D(1, 5, activation='relu')(in_image)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Flatten()(fe)
    fe = Dropout(0.4)(fe)

    # unsupervised discriminator model
    d_out_layer = Dense(1, activation='sigmoid')(fe)
    d_model = Model(in_image, d_out_layer)
    d_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

    # supervised discriminator model
    c_out_layer = Dense(2, activation='softmax')(fe)
    c_model = Model(in_image, c_out_layer)
    c_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])
    return d_model, c_model

In [55]:
d_model, c_model = define_discriminator()
d_model.summary()
c_model.summary()

Model: "model_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, 100, 25)           0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 96, 1)             126       
_________________________________________________________________
leaky_re_lu_32 (LeakyReLU)   (None, 96, 1)             0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 96)                0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 96)                0         
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 97        
Total params: 223
Trainable params: 223
Non-trainable params: 0
____________________________________________________________

# Generator

In [75]:
def define_generator():
    inshape = (100, 20)
    in_image = Input(shape=inshape)
    # todo: change to bidirectional
    fe = LSTM(25, activation='relu', return_sequences=True)(in_image)
    model = Model(in_image, fe)
    return model


# input_shape = (100, 1)

# def build_generator(input_shape, generator_reshape_dim=100, generator_reshape_shape=input_shape):
#     x = Input(shape=input_shape)
#     model = keras.models.Sequential()
#     model.add(keras.layers.Flatten())
#     model.add(keras.layers.Dense(units=generator_reshape_dim))
#     model.add(keras.layers.Reshape(target_shape=generator_reshape_shape))
#     model.add(keras.layers.Bidirectional(keras.layers.LSTM(units=64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode='concat'))
#     model.add(keras.layers.convolutional.UpSampling1D(size=1))
#     model.add(keras.layers.Bidirectional(keras.layers.LSTM(units=64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode='concat'))
#     model.add(keras.layers.TimeDistributed(keras.layers.Dense(units=25)))
#     model.add(keras.layers.Activation(activation='tanh'))
#     return Model(x, model(x))
              
# generator = build_generator(input_shape)
# generator.summary()

Model: "model_106"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_72 (InputLayer)        (None, 100, 1)            0         
_________________________________________________________________
sequential_12 (Sequential)   (None, 100, 25)           145933    
Total params: 145,933
Trainable params: 145,933
Non-trainable params: 0
_________________________________________________________________


In [77]:
def define_gan(g_model, d_model):
    
    # make weights in the discriminator not trainable
    d_model.trainable = False
    
    # connect image output from generator as input to discriminator
    gan_output = d_model(g_model.output)
    
    # define gan model as taking noise and outputting a classification
    model = Model(g_model.input, gan_output)
    
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [178]:
# select a supervised subset of the dataset, ensures classes are balanced
def select_supervised_samples(dataset, n_samples=100, n_classes=2):
    X, y = dataset
    X_list, y_list = list(), list()
    n_per_class = int(n_samples / n_classes)
    for i in range(n_classes):
        # get all images for this class
        X_with_class = X[y == i]
        # choose random instances
        ix = randint(0, len(X_with_class), n_per_class)
        # add to list
        [X_list.append(X_with_class[j]) for j in ix]
        [y_list.append(i) for j in ix]
    return asarray(X_list), asarray(y_list)

In [179]:
# select real samples
def generate_real_samples(dataset, n_samples):
    # split into images and labels
    images, labels = dataset
    # choose random instances
    ix = randint(0, images.shape[0], n_samples)
    # select images and labels
    X, labels = images[ix], labels[ix]
    # generate class labels
    y = ones((n_samples, 1))
    return [X, labels], y

# generate points in latent space as input for the generator
def generate_latent_points(n_samples):
    return np.array([np.random.normal(size=[100, 20]) for _ in range(n_samples)])
 
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, n_samples):
    # generate points in latent space
    z_input = generate_latent_points(n_samples)
    # predict outputs
    images = generator.predict(z_input)
    # create class labels
    y = zeros((n_samples, 1))
    return images, y

# Training

In [180]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, c_model, latent_dim, dataset, n_samples=100):
    # prepare fake examples
    X, _ = generate_fake_samples(g_model, n_samples)
    # scale from [-1,1] to [0,1]
    X = (X + 1) / 2.0
    # plot images
    for i in range(100):
        # define subplot
        pyplot.subplot(10, 10, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        # pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
    # save plot to file
    filename1 = 'generated_plot_%04d.png' % (step+1)
    pyplot.savefig(filename1)
    pyplot.close()
    # evaluate the classifier model
    X, y = dataset
    _, acc = c_model.evaluate(X, y, verbose=0)
    print('Classifier Accuracy: %.3f%%' % (acc * 100))
    # save the generator model
    filename2 = 'g_model_%04d.h5' % (step+1)
    g_model.save(filename2)
    # save the classifier model
    filename3 = 'c_model_%04d.h5' % (step+1)
    c_model.save(filename3)
    print('>Saved: %s, %s, and %s' % (filename1, filename2, filename3))

In [185]:
# train the generator and discriminator
def train(g_model, d_model, c_model, gan_model, dataset, latent_dim, n_epochs=20, n_batch=100):
    # select supervised dataset
    X_sup, y_sup = select_supervised_samples(dataset)
    print(X_sup.shape, y_sup.shape)
    
    # calculate the number of batches per training epoch
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    
    # calculate the size of half a batch of samples
    half_batch = int(n_batch / 2)
    print('n_epochs=%d, n_batch=%d, 1/2=%d, b/e=%d, steps=%d' % (n_epochs, n_batch, half_batch, bat_per_epo, n_steps))
    
    # manually enumerate epochs
    for i in range(n_steps):
        
        # update supervised discriminator (c)
        [Xsup_real, ysup_real], _ = generate_real_samples([X_sup, y_sup], half_batch)
        c_loss, c_acc = c_model.train_on_batch(Xsup_real, ysup_real)
        
        # update unsupervised discriminator (d)
        [X_real, _], y_real = generate_real_samples(dataset, half_batch)
        d_loss1 = d_model.train_on_batch(X_real, y_real)
        X_fake, y_fake = generate_fake_samples(g_model, half_batch)
        d_loss2 = d_model.train_on_batch(X_fake, y_fake)
        
        # update generator (g)
        X_gan, y_gan = generate_latent_points(n_batch), ones((n_batch, 1))
        g_loss = gan_model.train_on_batch(X_gan, y_gan)
        
        # summarize loss on this batch
        print('>%d, c[%.3f,%.0f], d[%.3f,%.3f], g[%.3f]' % (i+1, c_loss, c_acc*100, d_loss1, d_loss2, g_loss))
        
        # # evaluate the model performance every so often
        # if (i+1) % (bat_per_epo * 1) == 0:
        #     summarize_performance(i, g_model, c_model, latent_dim, dataset)


In [186]:
# size of the latent space
latent_dim = 100

# create the discriminator models
d_model, c_model = define_discriminator()

# create the generator
g_model = define_generator()

# create the gan
gan_model = define_gan(g_model, d_model)

# # load image data
dataset = [X_train, y_train]

# # train model
train(g_model, d_model, c_model, gan_model, dataset, latent_dim)

(100, 100, 25) (100,)
n_epochs=20, n_batch=100, 1/2=50, b/e=27, steps=540


/Users/lcwong/opt/anaconda3/envs/orion-env/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/Users/lcwong/opt/anaconda3/envs/orion-env/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>1, c[0.734,36], d[0.722,0.623], g[0.808]
>2, c[0.697,48], d[0.701,0.657], g[0.764]
>3, c[0.682,40], d[0.703,0.616], g[0.782]


/Users/lcwong/opt/anaconda3/envs/orion-env/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>4, c[0.697,52], d[0.700,0.641], g[0.792]
>5, c[0.714,54], d[0.703,0.637], g[0.804]
>6, c[0.696,56], d[0.700,0.596], g[0.781]
>7, c[0.713,52], d[0.698,0.591], g[0.805]
>8, c[0.694,52], d[0.702,0.633], g[0.811]
>9, c[0.692,56], d[0.701,0.589], g[0.798]
>10, c[0.692,50], d[0.711,0.605], g[0.839]
>11, c[0.705,50], d[0.704,0.612], g[0.794]
>12, c[0.699,56], d[0.691,0.609], g[0.815]
>13, c[0.716,42], d[0.691,0.623], g[0.809]
>14, c[0.701,42], d[0.711,0.625], g[0.845]
>15, c[0.694,56], d[0.698,0.641], g[0.805]
>16, c[0.697,48], d[0.692,0.584], g[0.832]
>17, c[0.698,58], d[0.694,0.597], g[0.849]
>18, c[0.687,44], d[0.694,0.577], g[0.848]
>19, c[0.690,42], d[0.691,0.569], g[0.849]
>20, c[0.696,48], d[0.689,0.588], g[0.848]
>21, c[0.678,68], d[0.695,0.592], g[0.833]
>22, c[0.690,48], d[0.691,0.588], g[0.816]
>23, c[0.684,58], d[0.695,0.598], g[0.846]
>24, c[0.686,54], d[0.692,0.627], g[0.846]
>25, c[0.691,44], d[0.689,0.595], g[0.797]
>26, c[0.699,40], d[0.696,0.612], g[0.860]
>27, c[0.698,48],

In [ ]:
dataset[1].shape